In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import nan

In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [4]:
data = pd.read_csv('household_power_consumption.txt', sep = ';',
                  parse_dates = True,
                  low_memory = False)

In [5]:
data.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [6]:
data['date_time'] = data['Date'].str.cat(data['Time'], sep= ' ')
data.drop(['Date', 'Time'], inplace= True, axis = 1)
data.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,date_time
0,4.216,0.418,234.840,18.400,0.000,1.000,17.0,16/12/2006 17:24:00
1,5.360,0.436,233.630,23.000,0.000,1.000,16.0,16/12/2006 17:25:00
2,5.374,0.498,233.290,23.000,0.000,2.000,17.0,16/12/2006 17:26:00
3,5.388,0.502,233.740,23.000,0.000,1.000,17.0,16/12/2006 17:27:00
4,3.666,0.528,235.680,15.800,0.000,1.000,17.0,16/12/2006 17:28:00


In [7]:
data.set_index(['date_time'], inplace=True)
data.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
date_time,,,,,,,
16/12/2006 17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
16/12/2006 17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
16/12/2006 17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
16/12/2006 17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
16/12/2006 17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [8]:
data.replace('?', nan, inplace=True)

In [9]:
data = data.astype('float')

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2075259 entries, 16/12/2006 17:24:00 to 26/11/2010 21:02:00
Data columns (total 7 columns):
Global_active_power      float64
Global_reactive_power    float64
Voltage                  float64
Global_intensity         float64
Sub_metering_1           float64
Sub_metering_2           float64
Sub_metering_3           float64
dtypes: float64(7)
memory usage: 126.7+ MB


In [11]:
np.isnan(data).sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64

In [12]:
def fill_missing(data):
    one_day = 24*60
    for row in range(data.shape[0]):
        for col in range(data.shape[1]):
            if np.isnan(data[row, col]):
                data[row, col] = data[row-one_day, col]

In [13]:
fill_missing(data.values)

In [14]:
np.isnan(data).sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2075259 entries, 16/12/2006 17:24:00 to 26/11/2010 21:02:00
Data columns (total 7 columns):
Global_active_power      float64
Global_reactive_power    float64
Voltage                  float64
Global_intensity         float64
Sub_metering_1           float64
Sub_metering_2           float64
Sub_metering_3           float64
dtypes: float64(7)
memory usage: 126.7+ MB


In [16]:
data.describe()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
count,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06
mean,1.089418e+00,1.236871e-01,2.408364e+02,4.618401e+00,1.118474e+00,1.291131e+00,6.448635e+00
std,1.054678e+00,1.125933e-01,3.240051e+00,4.433165e+00,6.141460e+00,5.796922e+00,8.433584e+00
min,7.600000e-02,0.000000e+00,2.232000e+02,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.080000e-01,4.800000e-02,2.389900e+02,1.400000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.020000e-01,1.000000e-01,2.410000e+02,2.600000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,1.526000e+00,1.940000e-01,2.428700e+02,6.400000e+00,0.000000e+00,1.000000e+00,1.700000e+01
max,1.112200e+01,1.390000e+00,2.541500e+02,4.840000e+01,8.800000e+01,8.000000e+01,3.100000e+01


In [17]:
data.shape

(2075259, 7)

In [18]:
data.to_csv('cleaned_data.csv')

In [19]:
dataset = pd.read_csv('cleaned_data.csv', parse_dates = True, index_col = 'date_time', low_memory = False)


In [20]:
dataset.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
date_time,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [21]:
data = dataset.resample('D').sum()

In [22]:
data.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
date_time,,,,,,,
2006-12-16,1209.176,34.922,93552.53,5180.8,0.0,546.0,4926.0
2006-12-17,3390.460,226.006,345725.32,14398.6,2033.0,4187.0,13341.0
2006-12-18,2203.826,161.792,347373.64,9247.2,1063.0,2621.0,14018.0
2006-12-19,1666.194,150.942,348479.01,7094.0,839.0,7602.0,6197.0
2006-12-20,2225.748,160.998,348923.61,9313.0,0.0,2648.0,14063.0


In [23]:
data_train = data.loc[:'2009-12-31', :]['Global_active_power']
data_train.head()

date_time
2006-12-16    1209.176
2006-12-17    3390.460
2006-12-18    2203.826
2006-12-19    1666.194
2006-12-20    2225.748
Freq: D, Name: Global_active_power, dtype: float64

In [24]:
data_test = data['2010']['Global_active_power']
data_test.head()

date_time
2010-01-01    1224.252
2010-01-02    1693.778
2010-01-03    1298.728
2010-01-04    1687.440
2010-01-05    1320.158
Freq: D, Name: Global_active_power, dtype: float64

In [25]:
data_train.shape

(1112,)

In [26]:
data_test.shape

(345,)

In [27]:
data_train.head(14)

date_time
2006-12-16    1209.176
2006-12-17    3390.460
2006-12-18    2203.826
2006-12-19    1666.194
2006-12-20    2225.748
2006-12-21    1723.288
2006-12-22    2341.338
2006-12-23    4773.386
2006-12-24    2550.012
2006-12-25    2743.120
2006-12-26    3934.110
2006-12-27    1528.760
2006-12-28    2072.638
2006-12-29    3174.392
Freq: D, Name: Global_active_power, dtype: float64

In [28]:
data_train = np.array(data_train)

In [29]:

X_train, y_train = [], []

for i in range(7, len(data_train)-7):
    X_train.append(data_train[i-7:i])
    y_train.append(data_train[i:i+7])

In [31]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [32]:
X_train.shape, y_train.shape

((1098, 7), (1098, 7))

In [35]:
pd.DataFrame(y_train).head()

,0,1,2,3,4,5,6
0,4773.386,2550.012,2743.120,3934.110,1528.760,2072.638,3174.392
1,2550.012,2743.120,3934.110,1528.760,2072.638,3174.392,2796.108
2,2743.120,3934.110,1528.760,2072.638,3174.392,2796.108,3494.196
3,3934.110,1528.760,2072.638,3174.392,2796.108,3494.196,2749.004
4,1528.760,2072.638,3174.392,2796.108,3494.196,2749.004,1824.760


In [36]:
x_scaler = MinMaxScaler()
X_train = x_scaler.fit_transform(X_train)

In [37]:
y_scaler = MinMaxScaler()
y_train = y_scaler.fit_transform(y_train)

In [38]:
pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6
0,0.211996,0.694252,0.431901,0.313037,0.436748,0.325660,0.462304
1,0.694252,0.431901,0.313037,0.436748,0.325660,0.462304,1.000000
2,0.431901,0.313037,0.436748,0.325660,0.462304,1.000000,0.508439
3,0.313037,0.436748,0.325660,0.462304,1.000000,0.508439,0.551133
4,0.436748,0.325660,0.462304,1.000000,0.508439,0.551133,0.814446


In [58]:
pd.DataFrame(X_test.reshape(331,7)).head()

,0,1,2,3,4,5,6
0,0.215329,0.319136,0.231795,0.317735,0.236533,0.225215,0.242910
1,0.319136,0.231795,0.317735,0.236533,0.225215,0.242910,0.067513
2,0.231795,0.317735,0.236533,0.225215,0.242910,0.067513,0.296925
3,0.317735,0.236533,0.225215,0.242910,0.067513,0.296925,0.277360
4,0.236533,0.225215,0.242910,0.067513,0.296925,0.277360,0.250466


In [39]:
X_train = X_train.reshape(1098, 7, 1)

In [60]:
pd.DataFrame(y_train)

,0,1,2,3,4,5,6
0,1.000000,0.608483,0.659577,0.974702,0.338269,0.482174,0.773688
1,0.508439,0.659577,0.974702,0.338269,0.482174,0.773688,0.673597
2,0.551133,0.974702,0.338269,0.482174,0.773688,0.673597,0.858305
3,0.814446,0.338269,0.482174,0.773688,0.673597,0.858305,0.661134
4,0.282652,0.482174,0.773688,0.673597,0.858305,0.661134,0.416588
...,...,...,...,...,...,...,...
1093,0.197623,0.472206,0.433326,0.318806,0.702646,0.496729,0.511538
1094,0.394568,0.433326,0.318806,0.702646,0.496729,0.511538,0.514669
1095,0.362081,0.318806,0.702646,0.496729,0.511538,0.514669,0.487852
1096,0.266389,0.702646,0.496729,0.511538,0.514669,0.487852,0.475647


## MODEL

In [40]:
reg = Sequential()
reg.add(LSTM(units = 200, activation = 'relu', input_shape=(7,1)))
reg.add(Dense(7))

In [41]:
reg.compile(loss='mse', optimizer='adam')

In [42]:
reg.fit(X_train, y_train, epochs = 100)

Train on 1098 samples
Epoch 1/100
1098/1098 [==============================] - 3s 3ms/sample - loss: 0.0604
Epoch 2/100
1098/1098 [==============================] - 0s 274us/sample - loss: 0.0251
Epoch 3/100
1098/1098 [==============================] - 0s 269us/sample - loss: 0.0242
Epoch 4/100
1098/1098 [==============================] - 0s 271us/sample - loss: 0.0240
Epoch 5/100
1098/1098 [==============================] - 0s 269us/sample - loss: 0.0240
Epoch 6/100
1098/1098 [==============================] - 0s 273us/sample - loss: 0.0239
Epoch 7/100
1098/1098 [==============================] - 0s 274us/sample - loss: 0.0242
Epoch 8/100
1098/1098 [==============================] - 0s 276us/sample - loss: 0.0239
Epoch 9/100
1098/1098 [==============================] - 0s 254us/sample - loss: 0.0238
Epoch 10/100
1098/1098 [==============================] - 0s 249us/sample - loss: 0.0238
Epoch 11/100
1098/1098 [==============================] - 0s 246us/sample - loss: 0.0239
Epoch 12/1

1098/1098 [==============================] - 0s 276us/sample - loss: 0.0232
Epoch 93/100
1098/1098 [==============================] - 0s 268us/sample - loss: 0.0230
Epoch 94/100
1098/1098 [==============================] - 0s 279us/sample - loss: 0.0229
Epoch 95/100
1098/1098 [==============================] - 0s 275us/sample - loss: 0.0231
Epoch 96/100
1098/1098 [==============================] - 0s 269us/sample - loss: 0.0229
Epoch 97/100
1098/1098 [==============================] - 0s 271us/sample - loss: 0.0228
Epoch 98/100
1098/1098 [==============================] - 0s 267us/sample - loss: 0.0234
Epoch 99/100
1098/1098 [==============================] - 0s 262us/sample - loss: 0.0229
Epoch 100/100
1098/1098 [==============================] - 0s 268us/sample - loss: 0.0230


## TEST

In [43]:
data_test = np.array(data_test)

In [44]:
X_test, y_test = [], []

for i in range(7, len(data_test)-7):
    X_test.append(data_test[i-7:i])
    y_test.append(data_test[i:i+7])

In [45]:

X_test, y_test = np.array(X_test), np.array(y_test)

In [46]:
X_test = x_scaler.transform(X_test)
y_test = y_scaler.transform(y_test)

In [47]:
X_test = X_test.reshape(331,7,1)

In [48]:
y_pred = reg.predict(X_test)

In [49]:
y_pred = y_scaler.inverse_transform(y_pred)

In [50]:
y_pred

array([[1404.5343 , 1408.0128 , 1468.2506 , ..., 1541.1553 , 1510.3818 ,
        1481.7954 ],
       [1057.0338 , 1191.8494 , 1325.114  , ..., 1459.3998 , 1440.8486 ,
        1420.4543 ],
       [1430.7891 , 1393.8259 , 1453.4998 , ..., 1473.2003 , 1436.5037 ,
        1415.1616 ],
       ...,
       [1274.3502 , 1044.4283 , 1055.2831 , ..., 1000.92175, 1003.24054,
         956.94305],
       [1654.1782 , 1396.8729 , 1242.709  , ..., 1178.6206 , 1142.1666 ,
        1131.79   ],
       [1624.2018 , 1388.6733 , 1222.6599 , ..., 1228.025  , 1220.4954 ,
        1234.009  ]], dtype=float32)

In [51]:
y_true = y_scaler.inverse_transform(y_test)

In [52]:
y_true

array([[ 555.664, 1593.318, 1504.82 , ...,    0.   , 1995.796, 2116.224],
       [1593.318, 1504.82 , 1383.18 , ..., 1995.796, 2116.224, 2196.76 ],
       [1504.82 , 1383.18 ,    0.   , ..., 2116.224, 2196.76 , 2150.112],
       ...,
       [1892.998, 1645.424, 1439.426, ..., 1973.382, 1109.574,  529.698],
       [1645.424, 1439.426, 2035.418, ..., 1109.574,  529.698, 1612.092],
       [1439.426, 2035.418, 1973.382, ...,  529.698, 1612.092, 1579.692]])

## Evaluate

In [53]:
def evaluate_model(y_true, y_predicted):
    scores = []
    
    #calculate scores for each day
    for i in range(y_true.shape[1]):
        mse = mean_squared_error(y_true[:, i], y_predicted[:, i])
        rmse = np.sqrt(mse)
        scores.append(rmse)
    
    #calculate score for whole prediction
    total_score = 0
    for row in range(y_true.shape[0]):
        for col in range(y_predicted.shape[1]):
            total_score = total_score + (y_true[row, col] - y_predicted[row, col])**2
    total_score = np.sqrt(total_score/(y_true.shape[0]*y_predicted.shape[1]))
    
    return total_score, scores

In [54]:

evaluate_model(y_true, y_pred)

(574.9018350292386,
 [576.8782722778151,
  586.3945946580905,
  572.3024810223732,
  565.9585568555709,
  579.9201354085528,
  573.1224599518497,
  569.4925305511084])

In [55]:
np.std(y_true[0])

710.0253857243853